In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 47.3 MB/s 
     |████████████████████████████████| 182 kB 60.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 81.5 MB/s 
     |████████████████████████████████| 115 kB 83.1 MB/s 
     |████████████████████████████████| 127 kB 77.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
foldername= "/content/drive/My Drive/nlpproject/"

In [ ]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

In [ ]:
class CFG:
    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    basic_lr=1e-3
    train = True
    debug = False
    offline = False
    models_path = "bert-base-uncased"
    epochs = 50
    save_all_models = False
    apex = True
    print_freq = 20
    num_workers = 4
    model = "bert-base-uncased"
    loss_func = 'SmoothL1'
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    llrd = True
    layerwise_lr = 5e-5
    layerwise_lr_decay = 0.9
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    pooling = 'mean' # mean, max, min, attention, weightedlayer
    layer_start = 4
    #init_weight
    init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True
    reinit_n = 1
    #adversarial
    fgm = False
    awp = False
    adv_lr = 1
    adv_eps = 0.2
    unscale = False
    eps = 1e-6
    betas = (0.9, 0.999)
    max_len = 100
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    target_cols = ['EI', 'SN', 'TF', 'JP']
    seed = 42
    cv_seed = 42
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 2048
    n_targets = 4
    gpu_id = 0
    device = f'cuda:{gpu_id}'
cfg=CFG()

In [ ]:

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained(cfg.model)
#the dataset class for the first dataset, tokenized, and labeled
class Ds_EI(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=0  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_SN(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=1  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_TF(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=2  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_JP(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=3  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
path=foldername+"dataset2.csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    #total_len=len(ds)
    #train_len=int(len(ds)*0.8)
    #val_len=int((total_len-train_len)/2)
    #test_len=total_len-train_len-val_len
    #[train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    #return DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
#NEED TO CHANGE WHEN SWITCH TASK
ds=Ds_EI(path, tokenizer)
dl_EI=getdl(ds)

In [ ]:
print(len(ds))

396523


In [ ]:
config = AutoConfig.from_pretrained(CFG.model, ouput_hidden_states = True)
model = AutoModel.from_pretrained(CFG.model, config=config)
model.requires_grads=False

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
device = torch.device(dev)
cpu=torch.device("cpu")
features=[]
labels=[]
model.to(device)
with torch.no_grad():
    for i,batch in enumerate(dl_EI):
        print(i*cfg.batch_size)
        batch.to(device)
        #last_hidden_layers=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"]).last_hidden_state[:,0]
        #last_hidden_layers.to("cpu")
        #features.extend(last_hidden_layers)
        output=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"])
        #print(output)
        output=output.last_hidden_state[:,0].cpu()
        #print(pooler_output[0])
        features.extend(output)
        l=batch["labels"].to("cpu")
        labels.extend(l)

0
128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048
2176
2304
2432
2560
2688
2816
2944
3072
3200
3328
3456
3584
3712
3840
3968
4096
4224
4352
4480
4608
4736
4864
4992
5120
5248
5376
5504
5632
5760
5888
6016
6144
6272
6400
6528
6656
6784
6912
7040
7168
7296
7424
7552
7680
7808
7936
8064
8192
8320
8448
8576
8704
8832
8960
9088
9216
9344
9472
9600
9728
9856
9984
10112
10240
10368
10496
10624
10752
10880
11008
11136
11264
11392
11520
11648
11776
11904
12032
12160
12288
12416
12544
12672
12800
12928
13056
13184
13312
13440
13568
13696
13824
13952
14080
14208
14336
14464
14592
14720
14848
14976
15104
15232
15360
15488
15616
15744
15872
16000
16128
16256
16384
16512
16640
16768
16896
17024
17152
17280
17408
17536
17664
17792
17920
18048
18176
18304
18432
18560
18688
18816
18944
19072
19200
19328
19456
19584
19712
19840
19968
20096
20224
20352
20480
20608
20736
20864
20992
21120
21248
21376
21504
21632
21760
21888
22016
22144
22272
22400
22528
22656
22784
22912
23040
231

In [ ]:
features=torch.stack(features)
labels=torch.stack(labels)

In [ ]:
torch.save(features, foldername+'EI_Extraced_feature_tensors_50*8000_lasthidden.pt')
torch.save(labels, foldername+'EI_Extraced_label_tensors_50*8000_lasthidden.pt')

In [ ]:
f=torch.load(foldername+'EI_Extraced_feature_tensors_50*8000_lasthidden.pt')

In [ ]:
(f[0]==features[0])

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr